In [35]:
#Let connect to the database and print the data from this folder 

import pandas as pd
import os


file_path = r"C:\Users\DeepakSureshNidagund\OneDrive - JA Solar GmbH\Documents - Sales Dashboards (BI Solution)\Y_EU Report\Europe Stock 最新版.xlsx"
    
try:
    # Check if file exists
    if not os.path.exists(file_path):
         raise FileNotFoundError(f"Excel file not found at: {file_path}")
        
    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name="Summary-Europe")
        
    # Print the first few rows of the data
    print("\nFirst 5 rows of the data:")
    print(df.head())
        
except Exception as e:
 print(f"An error occurred: {e}")

    
    





C:\Users\DeepakSureshNidagund\AppData\Roaming\Python\Python313\site-packages\openpyxl\worksheet\_reader.py:223: UserWarning: Cell O23763 is marked as a date but the serial value 2407090024 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)



First 5 rows of the data:
  DestinationWarehouse Factory Related transaction company  \
0      Rotterdam- KNNL      东台               JA Solar GmbH   
1      Rotterdam- KNNL      奉贤               JA Solar GmbH   
2      Rotterdam- KNNL      奉贤               JA Solar GmbH   
3      Rotterdam- KNNL      奉贤               JA Solar GmbH   
4      Rotterdam- Odin      奉贤               JA Solar GmbH   

  Related transaction Term                  EXW                  ETD  \
0                      CIF  2024-05-12 00:00:00                  NaN   
1                      CIF  2025-05-16 00:00:00  2025-05-21 00:00:00   
2                      CIF  2025-05-16 00:00:00  2025-05-21 00:00:00   
3                      CIF  2025-05-16 00:00:00  2025-05-21 00:00:00   
4                      CIF  2025-05-16 00:00:00  2025-05-22 00:00:00   

                   ETA update ETA  ATA         Date inbound  ...  EWX Week  \
0                  NaN        NaN  NaN  2024-07-09 00:00:00  ...  2024年05月   
1  2025-06-

In [36]:
df.columns

Index(['DestinationWarehouse', 'Factory', 'Related transaction company',
       'Related transaction Term', 'EXW', 'ETD', 'ETA', 'update ETA', 'ATA',
       'Date inbound', 'inbound status', 'B/L', 'Vessel', 'Sold Date', 'DN',
       'Inv.', 'Type', 'Type.1', 'Container', 'Qty(PC)', 'Qty(plts)',
       'Power(W)', 'Power(w/pc)', 'Status', 'Customer', 'Salesman',
       'Region info', 'Delivery Terms', 'InternalSalesContract',
       'External sales contract', 'Currency', 'Inv No.', 'C2 --> C1 Date',
       'Handover Date', 'Contractual Delivery Week', 'Country Code', '状态',
       'Internal related price', 'Battery type', 'Border Color',
       'Junction box', 'length', 'Voltage', 'Storage duration', 'Sold（Week）',
       'Storage duration（days）', 'original WH', 'Warehouse after transfer',
       'ITS', 'ITS-Remark', 'Identify Risk stock', 'ERP NO', 'ETD month',
       'Sold month', 'outbound quantity', 'Rest quantity',
       'Released on the sea', 'Booking No.', 'EWX Week', 'Type.2',
 

In [39]:

# 1. Remove special characters from specified columns
import re
def remove_special_chars(val):
    if pd.isnull(val):
        return val
    # Remove special characters and then strip leading/trailing spaces
    return re.sub(r'[^A-Za-z0-9 ]+', '', str(val)).strip()

columns_to_clean = ['Container', 'DN', 'Inv.']

for col in columns_to_clean:
    if col in df.columns:
        df[col] = df[col].apply(remove_special_chars)

# 2. Rename the columns
df = df.rename(columns={
    'Inv.': 'Invoice Number',
    'Container': 'Container Number',
    'DN': 'Release Number'
})

# 3. Remove specified columns
columns_to_remove = [
    'Factory', 'Related transaction company', 'Related transaction Term', 'Currency', 'Inv No.', 'C2 --> C1 Date',
    'Handover Date', 'Contractual Delivery Week', 'Country Code', '状态', 'Internal related price', 'Battery type',
    'Border Color', 'Junction box', 'length', 'Voltage', 'Storage duration', 'Sold（Week）', 'Storage duration（days）',
    'original WH', 'Warehouse after transfer', 'ETD month', 'Sold month', 'outbound quantity', 'Rest quantity',
    'Released on the sea', 'Booking No.', 'EWX Week', 'Type.2', 'Auxiliary column', '成品料号', 'Inv&type', '是否签收',
    '型号', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67'
]

# Drop the specified columns
df = df.drop(columns=columns_to_remove, errors='ignore')

# Create a new column 'Ref1' as concatenation of 'Release Number' and 'Container Number'
if 'Release Number' in df.columns and 'Container Number' in df.columns:
    df['Ref1'] = df['Release Number'].astype(str) + df['Container Number'].astype(str)
else:
    df['Ref1'] = None

df.columns


Index(['DestinationWarehouse', 'EXW', 'ETD', 'ETA', 'update ETA', 'ATA',
       'Date inbound', 'inbound status', 'B/L', 'Vessel', 'Sold Date',
       'Release Number', 'Invoice Number', 'Type', 'Type.1',
       'Container Number', 'Qty(PC)', 'Qty(plts)', 'Power(W)', 'Power(w/pc)',
       'Status', 'Customer', 'Salesman', 'Region info', 'Delivery Terms',
       'InternalSalesContract', 'External sales contract', 'ITS', 'ITS-Remark',
       'Identify Risk stock', 'ERP NO', 'Ref1'],
      dtype='object')

In [40]:
# Separate data into two datasets based on 'Release Number' and 'Not Released'
Released_data = df[(df['Release Number'].notna() & (df['Release Number'].astype(str).str.strip() != "")) & 
         (df['Sold Date'].notna() & (df['Sold Date'].astype(str).str.strip() != ""))]


Not_Released_data = df[(df['Release Number'].isna() | (df['Release Number'].astype(str).str.strip() == "")) & 
         (df['Sold Date'].isna() | (df['Sold Date'].astype(str).str.strip() == ""))]

In [ ]:
#Now Combinr the 'Released_data' with CDR data to remove the Outbound data from the Released_data




In [ ]:
Released_data.to_csv("Released_data.csv", index=False)
Not_Released_data.to_csv("Not_Released_data.csv", index=False)